In [65]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [66]:
df_sim = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/df_all_round_sim.csv')

columns required ::

'year', 'bowler_wkts', 'striker_runs', 'striker_balls',
       'non_striker_runs', 'non_striker_balls', 'bowler_balls', 'bowler_runs',
       ###########
       'last_18_legal_balls_run_rate',
       'last_18_legal_balls_wickets', 'last_18_legal_balls_4s',
       'last_18_legal_balls_6s'        

In [67]:
# striker_runs, non_striker_runs

striker_runs = []
non_striker_runs = []
i = 0


for match in df_sim.match_id.unique():
    i = i+1
    df_part = df_sim[df_sim.match_id==match]
    for inning in df_part.innings.unique():
        first_loc = df_part[df_part.innings==inning].index[0]

        for row in (df_part[df_part.innings==inning].index):
            s_r = df_part[df_part.innings==inning][df_part.striker==df_part.striker.loc[row]].loc[first_loc:row]\
                                                    .runs_off_bat.sum()
            n_s_r = df_part[df_part.innings==inning][df_part.striker==df_part.non_striker.loc[row]].loc[first_loc:row]\
                                                    .runs_off_bat.sum()
            striker_runs.append(s_r)
            non_striker_runs.append(n_s_r)

df_sim['striker_runs'] = striker_runs
df_sim['non_striker_runs'] = non_striker_runs

# striker_balls, non_striker_balls

striker_balls = []
non_striker_balls = []
i = 0


for match in df_sim.match_id.unique():
    i = i+1
    df_part = df_sim[df_sim.match_id==match]
    for inning in df_part.innings.unique():
        first_loc = df_part[df_part.innings==inning].index[0]

        for row in (df_part[df_part.innings==inning].index):
            s_r = df_part[df_part.innings==inning][df_part.striker==df_part.striker.loc[row]].loc[first_loc:row]\
                                                    .is_faced_by_batter.sum()
            n_s_r = df_part[df_part.innings==inning][df_part.striker==df_part.non_striker.loc[row]].loc[first_loc:row]\
                                                    .is_faced_by_batter.sum()
            striker_balls.append(s_r)
            non_striker_balls.append(n_s_r)

df_sim['striker_balls'] = striker_balls
df_sim['non_striker_balls'] = non_striker_balls

#bowler_runs, bowler_balls, bowler_wkts

#runs_conceeded, islegal, isBowlerWicket

df_sim['bowler_runs'] = df_sim.groupby(['match_id','innings','bowler'])['runs_conceeded'].cumsum()
df_sim['bowler_balls'] = df_sim.groupby(['match_id','innings','bowler'])['islegal'].cumsum()
df_sim['bowler_wkts'] = df_sim.groupby(['match_id','innings','bowler'])['isBowlerWicket'].cumsum()


/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_8695/3519308458.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  s_r = df_part[df_part.innings==inning][df_part.striker==df_part.striker.loc[row]].loc[first_loc:row]\
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_8695/3519308458.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  n_s_r = df_part[df_part.innings==inning][df_part.striker==df_part.non_striker.loc[row]].loc[first_loc:row]\
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_8695/3519308458.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  s_r = df_part[df_part.innings==inning][df_part.striker==df_part.striker.loc[row]].loc[first_loc:row]\
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_8695/3519308458.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  n_s_r = df_part[df_part.innings==inning][df_part

# inning 1

In [68]:
import numpy as np
import pandas as pd

m_id = 'M001'

live_score = df_sim[df_sim.match_id==m_id][df_sim.innings==1] #######

import joblib

# Load the regression model 'bst' and classification model 'lgbm_classifier'
final_model = joblib.load('/Users/roumyadas/IPL_regression_model.pkl')

live_score['year'] = 2025
live_score['partnership_runs'] = live_score['runs_scored']-live_score.last_fow
live_score.loc[live_score.wickets_down==0, 'last_fow'] = -1
live_score.loc[live_score.last_fow<0, 'partnership_runs'] = live_score.loc[live_score.last_fow<0, 'runs_scored']

# Initialize columns to store the metrics
live_score['last_18_legal_balls_run_rate'] = np.nan
live_score['last_18_legal_balls_wickets'] = np.nan
live_score['last_18_legal_balls_4s'] = np.nan
live_score['last_18_legal_balls_6s'] = np.nan

# Iterate through the DataFrame rows
for index, row in live_score.iterrows():
#for index, row in live_score[live_score.legal_balls_bowled>=36].iterrows():
    #match_id = row['match_id']
    #innings = row['innings']
    legal_balls_bowled = row['legal_balls_bowled']
    is_wicket = row['isWicket']
    is_four = row['isFour']
    is_six = row['isSix']

    # Calculate the start index for the last 18 legal balls
    index_to_get_of_ball = max(legal_balls_bowled-18+1, 1)
    start_index = live_score[live_score.legal_balls_bowled==index_to_get_of_ball].index[0]
    #start_index = max(index - legal_balls_bowled + 18, 0)

    # Filter the DataFrame for the relevant range
    relevant_data = live_score.loc[start_index:index]

    # Calculate the metrics for the last 18 legal balls
    run_rate = 6*relevant_data['total_runs'].sum() / relevant_data['islegal'].sum()
    wickets = relevant_data['isWicket'].sum()
    fours = relevant_data['isFour'].sum()
    sixes = relevant_data['isSix'].sum()

    # Update the corresponding columns in the original DataFrame
    live_score.at[index, 'last_18_legal_balls_run_rate'] = run_rate
    live_score.at[index, 'last_18_legal_balls_wickets'] = wickets
    live_score.at[index, 'last_18_legal_balls_4s'] = fours
    live_score.at[index, 'last_18_legal_balls_6s'] = sixes

#live_score = live_score[live_score.legal_balls_bowled>=18]

# Define the feature list
feature_list = [
    'year', 'wickets_down', 'striker_batting_position',
       'non_striker_batting_position', 'bowler_wkts', 'runs_scored',
       'run_rate', 'last_fow', 'striker_runs', 'striker_balls',
       'non_striker_runs', 'non_striker_balls', 'bowler_balls', 'bowler_runs',
       'last_fow_run_rate', 'fours', 'sixes', 'last_18_legal_balls_run_rate',
       'last_18_legal_balls_wickets', 'last_18_legal_balls_4s',
       'last_18_legal_balls_6s'
]

# Create a list of feature values in the correct order
features = np.array([live_score[feature_name] for feature_name in feature_list]).T

#pd.DataFrame(features)

# Perform regression prediction
regression_prediction = final_model.predict(features)

live_score['predicted_runs'] = regression_prediction

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_8695/2100321597.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  live_score = df_sim[df_sim.match_id==m_id][df_sim.innings==1] #######


In [69]:
np.round(live_score[['striker','non_striker','runs_off_bat','runs_scored','striker_runs','striker_balls',
         'non_striker_runs','non_striker_balls','bowler','bowler_runs','bowler_balls', 'bowler_wkts',
                    'predicted_runs']][live_score.legal_balls_bowled>18][live_score.legal_balls_bowled<=108],2)

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_8695/4233749886.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  np.round(live_score[['striker','non_striker','runs_off_bat','runs_scored','striker_runs','striker_balls',


,striker,non_striker,runs_off_bat,runs_scored,striker_runs,striker_balls,non_striker_runs,non_striker_balls,bowler,bowler_runs,bowler_balls,bowler_wkts,predicted_runs
18,RD Gaikwad,DP Conway,2,23,9,11,14,8,Rasikh Salam,10,7,0,182.48
19,RD Gaikwad,DP Conway,0,23,9,12,14,8,Rasikh Salam,10,8,0,182.93
20,RD Gaikwad,DP Conway,1,24,10,13,14,8,Rasikh Salam,11,9,0,182.93
21,DP Conway,RD Gaikwad,1,25,15,9,10,13,Rasikh Salam,12,10,0,182.95
22,RD Gaikwad,DP Conway,6,31,16,14,15,9,Rasikh Salam,18,11,0,184.51
23,RD Gaikwad,DP Conway,1,32,17,15,15,9,Rasikh Salam,19,12,0,184.46
24,RD Gaikwad,DP Conway,4,36,21,16,15,9,B Kumar,17,13,0,189.16
25,RD Gaikwad,DP Conway,1,37,22,17,15,9,B Kumar,18,14,0,188.91
26,DP Conway,RD Gaikwad,2,39,17,10,22,17,B Kumar,20,15,0,188.23
27,DP Conway,RD Gaikwad,1,40,18,11,22,17,B Kumar,21,16,0,188.50


# inning 2

In [70]:
import numpy as np
import pandas as pd

m_id = 'M001'

live_score = df_sim[df_sim.match_id==m_id][df_sim.innings==2] #######

import joblib

# Load the regression model 'bst' and classification model 'lgbm_classifier'
final_model = joblib.load('/Users/roumyadas/Desktop/Data/IPL 08-24 complete/IPL_win_pred_2nd_inn_classifier.joblib')

live_score['year'] = 2025
live_score['partnership_runs'] = live_score['runs_scored']-live_score.last_fow
live_score.loc[live_score.wickets_down==0, 'last_fow'] = -1
live_score.loc[live_score.last_fow<0, 'partnership_runs'] = live_score.loc[live_score.last_fow<0, 'runs_scored']

# Initialize columns to store the metrics
live_score['last_18_legal_balls_run_rate'] = np.nan
live_score['last_18_legal_balls_wickets'] = np.nan
live_score['last_18_legal_balls_4s'] = np.nan
live_score['last_18_legal_balls_6s'] = np.nan

# Iterate through the DataFrame rows
for index, row in live_score.iterrows():
#for index, row in live_score[live_score.legal_balls_bowled>=36].iterrows():
    #match_id = row['match_id']
    #innings = row['innings']
    legal_balls_bowled = row['legal_balls_bowled']
    is_wicket = row['isWicket']
    is_four = row['isFour']
    is_six = row['isSix']

    # Calculate the start index for the last 18 legal balls
    index_to_get_of_ball = max(legal_balls_bowled-18+1, 1)
    start_index = live_score[live_score.legal_balls_bowled==index_to_get_of_ball].index[0]
    #start_index = max(index - legal_balls_bowled + 18, 0)

    # Filter the DataFrame for the relevant range
    relevant_data = live_score.loc[start_index:index]

    # Calculate the metrics for the last 18 legal balls
    run_rate = 6*relevant_data['total_runs'].sum() / relevant_data['islegal'].sum()
    wickets = relevant_data['isWicket'].sum()
    fours = relevant_data['isFour'].sum()
    sixes = relevant_data['isSix'].sum()

    # Update the corresponding columns in the original DataFrame
    live_score.at[index, 'last_18_legal_balls_run_rate'] = run_rate
    live_score.at[index, 'last_18_legal_balls_wickets'] = wickets
    live_score.at[index, 'last_18_legal_balls_4s'] = fours
    live_score.at[index, 'last_18_legal_balls_6s'] = sixes

#live_score = live_score[live_score.legal_balls_bowled>=18]

# Define the feature list
feature_list = [
    'year', 'wickets_down', 
    'striker_batting_position','non_striker_batting_position', 
    'runs_scored', 'legal_balls_bowled',
       'run_rate', 
    'target', 'legal_balls_remaining', 'reqd_run_rate',
       'last_fow', 'partnership_runs', 
    'striker_runs', 'non_striker_runs',
       'last_fow_run_rate', 'fours', 'sixes', 
    'last_18_legal_balls_run_rate','last_18_legal_balls_wickets', 'last_18_legal_balls_4s','last_18_legal_balls_6s'
    , 'bowler_balls', 'bowler_runs', 'bowler_wkts'
]


#live_score = live_score[live_score.legal_balls_bowled>=18]

# Create a list of feature values in the correct order
features = np.array([live_score[feature_name] for feature_name in feature_list]).T

#pd.DataFrame(features)

# Perform regression prediction
win_prediction = final_model.predict_proba(features)

live_score['win_probability'] = win_prediction[:,1]

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_8695/3161240523.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  live_score = df_sim[df_sim.match_id==m_id][df_sim.innings==2] #######


In [71]:
np.round(live_score[['striker','non_striker','runs_off_bat','runs_scored','striker_runs','striker_balls',
         'non_striker_runs','non_striker_balls','bowler','bowler_runs','bowler_balls', 'bowler_wkts',
                    'win_probability']][live_score.legal_balls_bowled>18][live_score.legal_balls_bowled<=108],2)

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_8695/2689685228.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  np.round(live_score[['striker','non_striker','runs_off_bat','runs_scored','striker_runs','striker_balls',


,striker,non_striker,runs_off_bat,runs_scored,striker_runs,striker_balls,non_striker_runs,non_striker_balls,bowler,bowler_runs,bowler_balls,bowler_wkts,win_probability
142,V Kohli,PD Salt,4,41,17,11,23,9,R Ashwin,4,1,0,0.91
143,V Kohli,PD Salt,1,42,18,12,23,9,R Ashwin,5,2,0,0.91
144,PD Salt,V Kohli,4,46,27,10,18,12,R Ashwin,9,3,0,0.92
145,PD Salt,V Kohli,2,48,29,11,18,12,R Ashwin,11,4,0,0.92
146,JG Bethell,V Kohli,0,48,0,1,18,12,R Ashwin,11,5,0,0.92
147,JG Bethell,V Kohli,1,49,1,2,18,12,R Ashwin,12,6,0,0.92
148,JG Bethell,V Kohli,1,50,2,3,18,12,KK Ahmed,14,7,0,0.92
149,V Kohli,JG Bethell,2,52,20,13,2,3,KK Ahmed,16,8,0,0.92
150,V Kohli,JG Bethell,1,53,21,14,2,3,KK Ahmed,17,9,0,0.92
151,JG Bethell,V Kohli,1,54,3,4,21,14,KK Ahmed,18,10,0,0.92
